<a href="https://colab.research.google.com/github/remphanstar/WanBook/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Cell (1) WAN2GP Setup Introduction & Enhanced Platform Detection v4.0 (System Compliant)

import sys
import os
import subprocess
from IPython.display import display, HTML, Markdown

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Setup Introduction & Enhanced Platform Detection")

def detect_platform():
    """Enhanced platform detection with comprehensive indicators"""

    # Lightning AI detection - multiple indicators for reliability
    lightning_indicators = [
        "lightning" in str(sys.executable).lower(),
        "teamspace-studios" in os.getcwd(),
        "LIGHTNING_CLOUDSPACE_HOST" in os.environ,
        "LIGHTNING_CLOUDSPACE_ID" in os.environ,
        "commands/python" in str(sys.executable),
        "/home/zeus/miniconda3/envs/cloudspace" in str(sys.executable),
        os.path.exists("/teamspace"),
        os.path.exists("/commands")
    ]

    # Google Colab detection
    colab_indicators = [
        "google.colab" in sys.modules,
        "/content" in os.getcwd()
    ]

    # Vast.AI detection
    vast_indicators = [
        "VAST_CONTAINER_LABEL" in os.environ,
        "/workspace" in os.getcwd(),
        "vast" in os.environ.get("HOSTNAME", "").lower()
    ]

    if any(lightning_indicators):
        return "Lightning AI"
    elif any(colab_indicators):
        return "Google Colab"
    elif any(vast_indicators):
        return "Vast.AI/Generic"
    else:
        return "Vast.AI/Generic"

def get_platform_commands(platform):
    """Get platform-specific pip and python commands - ENFORCED VENV FOR COLAB"""
    if platform == "Lightning AI":
        return "pip", "python", False  # (pip_cmd, python_cmd, use_venv)
    elif platform == "Google Colab":
        # ENFORCED: Always use venv for Google Colab per system instructions
        return ".venv/bin/pip", ".venv/bin/python", True
    else:  # Vast.AI/Generic
        return ".venv/bin/pip", ".venv/bin/python", True

# Detect current platform
current_platform = detect_platform()
pip_cmd, python_cmd, use_venv = get_platform_commands(current_platform)

# Display platform information
display(HTML(f"""
<div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
           color: white; padding: 20px; border-radius: 10px; text-align: center; margin: 10px 0;
           box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
    <h1>🚀 Wan2GP Full Setup Notebook</h1>
    <p>A cross-platform Jupyter notebook (Colab, Lightning AI, Vast.ai) that installs Wan2GP,
       common LoRA packs, and optional performance extras (FlashAttention 2, SageAttention, xFormers).
       It accelerates all downloads with <strong>aria2c</strong> for maximum speed.</p>
</div>

<div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border: 1px solid #dee2e6; margin: 15px 0;">
    <div style="background-color: #007bff; color: white; padding: 15px; border-radius: 5px; text-align: center; margin-bottom: 20px;">
        <h3 style="margin: 0; color: white;">🎯 Platform Detection Results</h3>
        <p style="margin: 5px 0 0 0; color: white;">Currently Running on: <strong style="color: #28a745; background-color: white; padding: 2px 6px; border-radius: 3px;">{current_platform}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Virtual Environment: <strong>{"Yes" if use_venv else "No (Lightning AI)"}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Pip Command: <strong>{pip_cmd}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Python Command: <strong>{python_cmd}</strong></p>
    </div>
</div>
"""))

print("📋 Platform Detection Debug Info:")
print(f"🔍 - Detected Platform: {current_platform}")
print(f"🐍 - Python Executable: {sys.executable}")
print(f"📁 - Current Working Directory: {os.getcwd()}")
print(f"🌐 - Google Colab Check: {'google.colab' in sys.modules}")
print(f"⚡ - Lightning Environment Variables: {[key for key in os.environ.keys() if 'LIGHTNING' in key]}")
print(f"🔧 - Virtual Environment Usage: {use_venv}")
print(f"📦 - Pip Command: {pip_cmd}")
print(f"🐍 - Python Command: {python_cmd}")

# CRITICAL: Enforce venv for Google Colab per system instructions
if current_platform == "Google Colab" and not use_venv:
    print("🚨 CRITICAL: Detected Google Colab but venv is disabled - FORCE ENABLING")
    use_venv = True
    pip_cmd = ".venv/bin/pip"
    python_cmd = ".venv/bin/python"
    print("✅ ENFORCED: Google Colab now using venv commands")
elif current_platform == "Google Colab":
    print("✅ CONFIRMED: Google Colab correctly configured for venv usage")


📋 Platform Detection Debug Info:
🔍 - Detected Platform: Google Colab
🐍 - Python Executable: /usr/bin/python3
📁 - Current Working Directory: /content
🌐 - Google Colab Check: True
⚡ - Lightning Environment Variables: []
🔧 - Virtual Environment Usage: True
📦 - Pip Command: .venv/bin/pip
🐍 - Python Command: .venv/bin/python
✅ CONFIRMED: Google Colab correctly configured for venv usage


In [2]:
#@title Cell (2) Complete System + Python Environment Setup - Cross-Platform v4.0 (System Compliant)

import sys
import os
import subprocess
import shutil
import time
from pathlib import Path

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        from IPython.display import display, Markdown
        display(Markdown(f"## {title}"))

jupyter_lab_title("Complete System + Python Environment Setup")

class CompleteEnvironmentSetup:
    def __init__(self):
        # Use global platform configuration from Cell 1
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        # CRITICAL ENFORCEMENT: Ensure venv for Google Colab per system instructions
        self.enforce_colab_venv()

        self.setup_phases = [
            ("System Packages", self.install_system_packages),
            ("Directory Protection", self.setup_directory_protection),
            ("Repository Clone", self.clone_repository),
            ("Virtual Environment", self.setup_virtual_environment),
            ("Matplotlib Fix", self.fix_matplotlib_backend),
            ("PyTorch Installation", self.install_pytorch),
            ("Requirements", self.install_requirements),
            ("Environment Verification", self.verify_complete_setup),
        ]

        self.current_phase = 0
        self.total_phases = len(self.setup_phases)

    def enforce_colab_venv(self):
        """CRITICAL: Enforce venv usage for Google Colab per system instructions"""
        if hasattr(self, 'current_platform') and self.current_platform == "Google Colab":
            if not hasattr(self, 'use_venv') or not self.use_venv or self.pip_cmd != ".venv/bin/pip":
                print("🚨 ENFORCING: Google Colab MUST use venv per system instructions")
                self.use_venv = True
                self.pip_cmd = ".venv/bin/pip"
                self.python_cmd = ".venv/bin/python"
                print("✅ ENFORCED: System setup using venv commands")
            else:
                print("✅ VERIFIED: System setup correctly using venv for Colab")

    def detect_platform_fallback(self):
        """Fallback platform detection - ENFORCES VENV FOR COLAB"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            # ENFORCED: Always venv for Colab per system instructions
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def update_progress(self, phase_name, status="in_progress"):
        """Update setup progress with visual indicators"""
        progress_percent = (self.current_phase / self.total_phases) * 100
        status_icons = {
            "in_progress": "🔄",
            "success": "✅",
            "failed": "❌"
        }

        print(f"\n[{self.current_phase + 1}/{self.total_phases}] {status_icons[status]} {phase_name}")
        print(f"Progress: {progress_percent:.1f}% | Platform: {self.current_platform}")
        print(f"🔧 Using: {self.pip_cmd} | {self.python_cmd} | venv: {self.use_venv}")
        print("=" * 60)

    def run_command_safely(self, command, description, timeout=300):
        """Execute command with comprehensive error handling"""
        try:
            result = subprocess.run(
                command,
                capture_output=True,
                text=True,
                timeout=timeout
            )

            if result.returncode == 0:
                print(f"✅ {description} completed successfully")
                return True, result.stdout
            else:
                print(f"❌ {description} failed (exit code {result.returncode})")
                if result.stderr:
                    # Check for specific ensurepip failure
                    if "ensurepip" in result.stderr:
                        print(f"⚠️ ensurepip module failure detected")
                        return "ensurepip_failure", result.stderr
                    print(f"   Error: {result.stderr.strip()[:200]}")
                return False, result.stderr

        except subprocess.TimeoutExpired:
            print(f"⏰ {description} timed out after {timeout}s")
            return False, f"Timeout after {timeout}s"
        except Exception as e:
            print(f"💥 {description} crashed: {e}")
            return False, str(e)

    def install_system_packages(self):
        """Install system packages - same for all platforms"""
        print("🔧 Installing system packages...")

        if not shutil.which("apt-get"):
            print("📦 apt-get not found. Skipping system package installation.")
            return True

        try:
            print("📦 Updating package lists...")
            subprocess.run(["sudo", "apt-get", "update", "-qq"], check=True, timeout=60)

            print("🔧 Installing aria2, git, build-essential, and wget...")
            subprocess.run(["sudo", "apt-get", "install", "-y", "aria2", "git", "build-essential", "wget"], check=True, timeout=180)

            print("✅ System packages installed successfully")
            return True
        except subprocess.CalledProcessError as e:
            print(f"❌ System package installation failed: {e}")
            return False
        except subprocess.TimeoutExpired:
            print("⏰ System package installation timed out")
            return False

    def setup_directory_protection(self):
        """Critical directory protection to prevent recursive cloning"""
        print("🛡️ Setting up directory protection...")

        current_dir = os.getcwd()
        print(f"📍 Current directory: {current_dir}")

        # Check if we're already inside a Wan2GP directory
        if "Wan2GP" in current_dir:
            print("⚠️ Already inside Wan2GP directory - navigating to safe location")

            # Navigate to content root (Colab) or appropriate base directory
            if self.current_platform == "Google Colab":
                safe_dir = "/content"
            elif self.current_platform == "Lightning AI":
                safe_dir = os.path.expanduser("~")
            else:
                safe_dir = os.path.expanduser("~")

            try:
                os.chdir(safe_dir)
                print(f"📁 Moved to safe directory: {os.getcwd()}")
            except Exception as e:
                print(f"❌ Failed to navigate to safe directory: {e}")
                return False

        # Check for existing Wan2GP directory and handle it
        repo_path = os.path.join(os.getcwd(), "Wan2GP")
        if os.path.exists(repo_path):
            print(f"📁 Found existing Wan2GP directory at: {repo_path}")

            # Check if it looks like a valid repository
            if os.path.exists(os.path.join(repo_path, ".git")):
                print("✅ Existing directory appears to be a valid git repository")
                # Check if it's the correct repository
                try:
                    os.chdir(repo_path)
                    result = subprocess.run(["git", "remote", "get-url", "origin"],
                                          capture_output=True, text=True, timeout=10)
                    if result.returncode == 0 and "Wan2GP" in result.stdout:
                        print("✅ Existing repository is correct - using it")
                        return True
                    else:
                        print("⚠️ Existing repository is not the correct one")
                        os.chdir("..")
                except Exception as e:
                    print(f"⚠️ Could not verify existing repository: {e}")
                    os.chdir("..")

            # If we reach here, remove the problematic directory
            print("🗑️ Removing problematic existing directory...")
            try:
                shutil.rmtree(repo_path)
                print("✅ Problematic directory removed")
            except Exception as e:
                print(f"❌ Failed to remove existing directory: {e}")
                # Try to rename it instead
                try:
                    backup_name = f"Wan2GP_backup_{int(time.time())}"
                    os.rename(repo_path, backup_name)
                    print(f"✅ Renamed problematic directory to {backup_name}")
                except Exception as e2:
                    print(f"❌ Could not even rename directory: {e2}")
                    return False

        print("✅ Directory protection setup complete")
        return True

    def clone_repository(self):
        """Clone WAN2GP repository with enhanced protection"""
        REPO_DIR = "Wan2GP"
        REPO_URL = "https://github.com/deepbeepmeep/Wan2GP.git"

        print("📂 Setting up WAN2GP repository...")
        print(f"📍 Working from: {os.getcwd()}")

        # Final check - ensure we're not in a nested situation
        current_path = os.getcwd()
        if current_path.count("Wan2GP") > 0:
            print("🚨 CRITICAL: Still in Wan2GP directory path!")
            print("🔄 Attempting emergency navigation...")

            # Emergency navigation
            if self.current_platform == "Google Colab":
                emergency_path = "/content"
            else:
                emergency_path = os.path.expanduser("~")

            try:
                os.chdir(emergency_path)
                print(f"🆘 Emergency navigation successful: {os.getcwd()}")
            except Exception as e:
                print(f"💥 Emergency navigation failed: {e}")
                return False

        if not os.path.exists(REPO_DIR):
            print(f"📥 Cloning repository from {REPO_URL}...")
            try:
                subprocess.run(["git", "clone", "--depth", "1", REPO_URL], check=True, timeout=120)
                print("✅ Repository cloned successfully")
            except subprocess.CalledProcessError as e:
                print(f"❌ Failed to clone repository: {e}")
                return False
            except subprocess.TimeoutExpired:
                print("⏰ Repository clone timed out")
                return False
        else:
            print(f"📁 Directory {REPO_DIR} already exists")

        try:
            os.chdir(REPO_DIR)
            final_path = os.getcwd()
            print(f"📁 Changed to directory: {final_path}")

            # Verify we're in the right place
            if final_path.count("Wan2GP") > 1:
                print("🚨 WARNING: Detected nested Wan2GP directories!")
                print(f"🔍 Current path: {final_path}")
                return False

            print("✅ Repository setup verified")
            return True
        except FileNotFoundError:
            print(f"❌ Directory {REPO_DIR} not found")
            return False

    def setup_venv_manual_pip(self):
        """Create venv without pip, then install pip manually"""
        try:
            print("🔧 Creating venv without pip...")
            subprocess.run([sys.executable, "-m", "venv", ".venv", "--without-pip"],
                          check=True, timeout=60)

            print("📥 Downloading get-pip.py...")
            subprocess.run(["wget", "https://bootstrap.pypa.io/get-pip.py"], check=True)

            print("🔧 Installing pip manually...")
            subprocess.run([".venv/bin/python", "get-pip.py"], check=True)

            print("✅ Virtual environment with manual pip created successfully")
            return True
        except Exception as e:
            print(f"❌ Manual pip installation failed: {e}")
            return False

    def setup_virtual_environment(self):
        """Create virtual environment - ENFORCED for Google Colab"""
        if not self.use_venv:
            print("⚡ Lightning AI detected - using system environment (no venv)")
            return True

        print("🐍 Setting up virtual environment...")
        print(f"🔧 Platform: {self.current_platform} - venv REQUIRED per system instructions")

        # Check if venv already exists
        if os.path.exists(".venv/bin/python"):
            print("✅ Virtual environment already exists and appears functional")
            return True

        print("🔨 Creating virtual environment...")

        # Try multiple methods for virtual environment creation
        venv_methods = [
            ([sys.executable, "-m", "venv", "--system-site-packages", ".venv"], "venv with system packages"),
            ([sys.executable, "-m", "venv", ".venv"], "standard venv"),
            ([sys.executable, "-m", "virtualenv", ".venv"], "virtualenv package"),
        ]

        for command, method_name in venv_methods:
            print(f"🔄 Trying: {method_name}")
            success, error_msg = self.run_command_safely(command, f"Virtual environment creation ({method_name})", timeout=60)

            if success == True:
                print("✅ Virtual environment created successfully")
                return True
            elif success == "ensurepip_failure":
                print("⚠️ ensurepip failure detected - trying manual pip installation...")
                if self.setup_venv_manual_pip():
                    return True
                break  # Don't try other methods if ensurepip failed
            else:
                print(f"❌ {method_name} failed")
                if error_msg:
                    print(f"   Error: {error_msg[:200]}")
                continue

        # Final fallback ONLY for non-Colab platforms per system instructions
        if self.current_platform != "Google Colab":
            print("🔄 All virtual environment methods failed - switching to system installation")
            print("⚡ Overriding to system-wide installation")
            self.use_venv = False
            self.pip_cmd = "pip"
            self.python_cmd = "python"
            return True
        else:
            print("🚨 CRITICAL: Google Colab REQUIRES venv per system instructions - cannot fallback")
            return False

    def fix_matplotlib_backend(self):
        """Fix matplotlib backend issue for Colab environment"""
        print("🎨 Fixing matplotlib backend compatibility...")

        try:
            # Create matplotlib configuration fix
            matplotlib_fix_script = """
import os
import warnings

# Fix matplotlib backend for Colab/Jupyter environments
# This prevents the 'module://matplotlib_inline.backend_inline' error
def fix_matplotlib_backend():
    # Override problematic MPLBACKEND environment variable
    if 'MPLBACKEND' in os.environ:
        old_backend = os.environ['MPLBACKEND']
        if 'inline' in old_backend or 'module://' in old_backend:
            # Set to a safe backend for headless environments
            os.environ['MPLBACKEND'] = 'Agg'
            print(f"🔧 Fixed matplotlib backend: {old_backend} -> Agg")

    # Ensure matplotlib uses safe backend before any imports
    try:
        import matplotlib
        matplotlib.use('Agg', force=True)
        print("✅ Matplotlib backend set to 'Agg' (headless mode)")
    except ImportError:
        print("⚠️ Matplotlib not yet installed - will be fixed after requirements")

    return True

# Apply the fix
fix_matplotlib_backend()
"""

            # Write the fix to a file that can be imported
            fix_file_path = "matplotlib_backend_fix.py"
            with open(fix_file_path, 'w') as f:
                f.write(matplotlib_fix_script)

            # Execute the fix
            success, _ = self.run_command_safely([self.python_cmd, fix_file_path],
                                               "Matplotlib backend fix", timeout=30)

            if success:
                print("✅ Matplotlib backend fix applied successfully")
                return True
            else:
                print("⚠️ Matplotlib backend fix execution failed, continuing anyway")
                return True  # Non-critical failure

        except Exception as e:
            print(f"⚠️ Matplotlib backend fix failed: {e}")
            print("🔄 Continuing setup - will be resolved during runtime")
            return True  # Non-critical failure

    def install_pytorch(self):
        """Install PyTorch using enforced venv commands"""
        print("🔥 Installing PyTorch...")
        print(f"🔧 Using pip command: {self.pip_cmd}")

        # Use CUDA 12.4 for RTX 10XX-40XX as per documentation[6]
        pytorch_cmd = [
            self.pip_cmd, "install",
            "torch==2.6.0", "torchvision", "torchaudio",
            "--index-url", "https://download.pytorch.org/whl/test/cu124"
        ]

        success, _ = self.run_command_safely(pytorch_cmd, "PyTorch installation", timeout=300)
        return success == True

    def install_requirements(self):
        """Install requirements.txt using enforced venv commands"""
        print("📦 Installing requirements...")
        print(f"🔧 Using pip command: {self.pip_cmd}")

        if not os.path.exists("requirements.txt"):
            print("⚠️ requirements.txt not found, skipping")
            return True

        req_cmd = [self.pip_cmd, "install", "-r", "requirements.txt"]
        success, _ = self.run_command_safely(req_cmd, "Requirements installation", timeout=300)
        return success == True

    def verify_complete_setup(self):
        """Verify the complete installation using enforced venv commands"""
        print("🔍 Verifying installation...")
        print(f"🔧 Using python command: {self.python_cmd}")

        # Verify directory structure
        current_dir = os.getcwd()
        print(f"📍 Final directory: {current_dir}")

        if current_dir.count("Wan2GP") > 1:
            print("🚨 CRITICAL: Nested directory structure detected!")
            print("❌ Setup verification failed due to directory structure")
            return False

        try:
            # Test PyTorch installation
            result = subprocess.run([self.python_cmd, "-c", "import torch; print(f'PyTorch {torch.__version__} installed'); print(f'CUDA available: {torch.cuda.is_available()}')"],
                                  capture_output=True, text=True, timeout=30)

            if result.returncode == 0:
                print("✅ PyTorch verification successful:")
                for line in result.stdout.strip().split('\n'):
                    print(f"   {line}")

                # Verify main WanGP file exists
                if os.path.exists("wgp.py"):
                    print("✅ WanGP main file found")
                else:
                    print("⚠️ WanGP main file (wgp.py) not found")

                return True
            else:
                print(f"❌ PyTorch verification failed: {result.stderr}")
                return False

        except Exception as e:
            print(f"❌ Verification failed: {e}")
            return False

    def run_complete_setup(self):
        """Execute the complete setup process with enforced venv"""
        print("🚀 WAN2GP Complete System + Python Environment Setup")
        print("=" * 70)
        print(f"Platform: {self.current_platform}")
        print(f"Virtual Environment: {'Yes' if self.use_venv else 'No'}")
        print(f"Pip Command: {self.pip_cmd}")
        print(f"Python Command: {self.python_cmd}")
        print("=" * 70)

        # Final enforcement check per system instructions
        if hasattr(self, 'current_platform') and self.current_platform == "Google Colab" and not self.use_venv:
            print("🚨 FATAL: Google Colab not using venv per system instructions - ABORTING")
            return False

        start_time = time.time()

        for phase_name, phase_func in self.setup_phases:
            self.update_progress(phase_name, "in_progress")

            phase_start = time.time()
            success = phase_func()
            phase_duration = time.time() - phase_start

            if success:
                self.update_progress(phase_name, "success")
                print(f"⏱️ {phase_name} completed in {phase_duration:.1f}s")
                self.current_phase += 1
            else:
                self.update_progress(phase_name, "failed")
                print(f"💥 Setup failed at: {phase_name}")
                print("\n🔄 Setup Issues - Platform-Specific Troubleshooting:")
                print("📱 Colab: Restart runtime and re-run all cells")
                print("⚡ Lightning.AI: Ensure you're using the correct Python version")
                print("🌌 Vast.AI: Check GPU drivers and CUDA installation")
                return False

        total_duration = time.time() - start_time
        print(f"\n🎉 WAN2GP setup completed successfully in {total_duration:.1f}s!")
        print("🚀 Ready to launch WAN2GP!")
        return True

# Execute the complete setup
setup = CompleteEnvironmentSetup()
setup.run_complete_setup()


✅ VERIFIED: System setup correctly using venv for Colab
🚀 WAN2GP Complete System + Python Environment Setup
Platform: Google Colab
Virtual Environment: Yes
Pip Command: .venv/bin/pip
Python Command: .venv/bin/python

[1/8] 🔄 System Packages
Progress: 0.0% | Platform: Google Colab
🔧 Using: .venv/bin/pip | .venv/bin/python | venv: True
🔧 Installing system packages...
📦 Updating package lists...
🔧 Installing aria2, git, build-essential, and wget...
✅ System packages installed successfully

[1/8] ✅ System Packages
Progress: 0.0% | Platform: Google Colab
🔧 Using: .venv/bin/pip | .venv/bin/python | venv: True
⏱️ System Packages completed in 11.2s

[2/8] 🔄 Directory Protection
Progress: 12.5% | Platform: Google Colab
🔧 Using: .venv/bin/pip | .venv/bin/python | venv: True
🛡️ Setting up directory protection...
📍 Current directory: /content
📁 Found existing Wan2GP directory at: /content/Wan2GP
✅ Existing directory appears to be a valid git repository
✅ Existing repository is correct - using it



True

In [3]:
#@title Cell (3) WAN2GP Essential Downloads - Complete Interface Population System

import os
import subprocess
import requests
from pathlib import Path
import time
import urllib.request
from IPython.display import display, HTML

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        from IPython.display import display, Markdown
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Essential Downloads - Complete Interface Population System")

class WAN2GPCompleteDownloader:
    def __init__(self):
        # Use global platform configuration from Cell 1
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        # Essential model configurations from documentation[3][14]
        self.essential_models = {
            # Core Text2Video Models - populate model dropdown
            "wan_t2v_1_3b": {
                "name": "Wan 2.1 Text2Video 1.3B",
                "filename": "Wan2_1-T2V-1_3B_bf16.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-1_3B_bf16.safetensors",
                "size": "2.87 GB",
                "destination": "ckpts/",
                "priority": "essential",
                "description": "Fast T2V model (6GB VRAM)"
            },
            "wan_t2v_1_3b_fp32": {
                "name": "Wan 2.1 Text2Video 1.3B FP32",
                "filename": "Wan2_1-T2V-1_3B_fp32.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-1_3B_fp32.safetensors",
                "size": "5.68 GB",
                "destination": "ckpts/",
                "priority": "high",
                "description": "I2V compatibility model"
            },
            # Essential Text Encoder - required for all models[3]
            "umt5_encoder": {
                "name": "UMT5 Text Encoder XXL",
                "filename": "umt5_xxl_fp16.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/umt5_xxl_fp16.safetensors",
                "size": "3.0 GB",
                "destination": "text_encoders/",
                "priority": "essential",
                "description": "Required text encoder for all models"
            },
            # Essential VAE Models - required for generation[3]
            "wan_vae_bf16": {
                "name": "Wan VAE BF16",
                "filename": "Wan2_1_VAE_bf16.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1_VAE_bf16.safetensors",
                "size": "254 MB",
                "destination": "vae/",
                "priority": "essential",
                "description": "Required VAE for video generation"
            },
            "wan_vae_fp32": {
                "name": "Wan VAE FP32",
                "filename": "Wan2_1_VAE_fp32.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1_VAE_fp32.safetensors",
                "size": "508 MB",
                "destination": "vae/",
                "priority": "high",
                "description": "Alternative VAE format"
            }
        }

        # Essential LoRA configurations from documentation[9][17]
        self.essential_loras = {
            # Speed Enhancement LoRAs - populate LoRA checkboxes[9]
            "safe_forcing": {
                "name": "Safe-Forcing lightx2v (2-step generation)",
                "filename": "Wan2_1-T2V-14B-lightx2v-cfg-step-distill-lora-rank32.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-14B-lightx2v-cfg-step-distill-lora-rank32.safetensors",
                "size": "134 MB",
                "destination": "loras/",
                "priority": "essential",
                "description": "2-step generation, 2x speed boost"
            },
            "causvid": {
                "name": "CausVid (4-12 step generation)",
                "filename": "Wan2_1-CausVid-14B-T2V-lora-rank32.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-CausVid-14B-T2V-lora-rank32.safetensors",
                "size": "134 MB",
                "destination": "loras/",
                "priority": "essential",
                "description": "4-12 steps, 2x speed improvement"
            },
            "accvid_t2v": {
                "name": "AccVid T2V (2x speed improvement)",
                "filename": "Wan2_1-AccVid-T2V-14B-lora-rank32-fp16.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-AccVid-T2V-14B-lora-rank32-fp16.safetensors",
                "size": "134 MB",
                "destination": "loras/",
                "priority": "high",
                "description": "2x speed, no CFG needed"
            },
            "accvid_i2v": {
                "name": "AccVid I2V (Image2Video acceleration)",
                "filename": "Wan2_1-AccVid-I2V-480P-14B-lora-rank32-fp16.safetensors",
                "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-AccVid-I2V-480P-14B-lora-rank32-fp16.safetensors",
                "size": "134 MB",
                "destination": "lorasi2v/",
                "priority": "high",
                "description": "I2V speed enhancement"
            }
        }

        self.all_downloads = {**self.essential_models, **self.essential_loras}

    def detect_platform_fallback(self):
        """Fallback platform detection - ENFORCES VENV FOR COLAB per system instructions"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def create_directory_structure(self):
        """Create complete WAN2GP directory structure from documentation[3][9]"""
        print("📁 Creating WAN2GP directory structure...")

        # Complete directory structure from documentation[9][17]
        directories = {
            "ckpts": "Main model files - populate model dropdown",
            "text_encoders": "Text encoding models - required for generation",
            "vae": "VAE models - required for generation",
            "vae_approx": "Approximate VAE models",
            "clip_vision": "CLIP vision models",
            "loras": "General T2V LoRAs - populate LoRA checkboxes",
            "loras1.3B": "1.3B specific LoRAs",
            "loras14B": "14B specific LoRAs",
            "lorasi2v": "Image-to-video LoRAs",
            "lorashunyuan": "Hunyuan Video LoRAs",
            "lorashunyuani2v": "Hunyuan I2V LoRAs",
            "lorasltxv": "LTX Video LoRAs"
        }

        for directory, description in directories.items():
            Path(directory).mkdir(parents=True, exist_ok=True)
            print(f"   ✅ {directory}/ - {description}")

        print("✅ Complete directory structure created")
        return True

    def download_with_fallback_methods(self, config, item_id):
        """Download using multiple methods with comprehensive fallback"""
        filename = config["filename"]
        url = config["url"]
        destination = config["destination"]
        full_path = os.path.join(destination, filename)

        print(f"\n📥 Downloading: {config['name']}")
        print(f"📊 Size: {config['size']}")
        print(f"📄 File: {filename}")
        print(f"📁 Destination: {full_path}")
        print(f"🔗 URL: {url}")

        # Check if file already exists
        if os.path.exists(full_path):
            file_size = os.path.getsize(full_path)
            if file_size > 1024:  # File exists and has content
                print(f"✅ {filename} already exists ({file_size:,} bytes) - skipping")
                return True

        # Ensure destination directory exists
        Path(destination).mkdir(parents=True, exist_ok=True)

        # Method 1: HuggingFace Hub (if available)
        if self.try_huggingface_hub_download(url, full_path, filename):
            return True

        # Method 2: aria2c (fastest for large files)
        if self.try_aria2c_download(url, full_path):
            return True

        # Method 3: wget (reliable fallback)
        if self.try_wget_download(url, full_path):
            return True

        # Method 4: Python urllib (final fallback)
        if self.try_urllib_download(url, full_path):
            return True

        print(f"❌ All download methods failed for {filename}")
        return False

    def try_huggingface_hub_download(self, url, full_path, filename):
        """Try HuggingFace Hub download using python command"""
        try:
            print("🔄 Trying HuggingFace Hub...")

            if "huggingface.co" in url and "/resolve/main/" in url:
                repo_part = url.split("huggingface.co/")[1].split("/resolve/main/")[0]
                file_part = url.split("/resolve/main/")[1]

                # Create download script
                download_script = f'''
import sys
try:
    from huggingface_hub import hf_hub_download
    import os

    # Download to proper location
    downloaded_path = hf_hub_download(
        repo_id="{repo_part}",
        filename="{file_part}",
        local_dir=".",
        local_dir_use_symlinks=False
    )

    # Verify file exists and has content
    if os.path.exists("{full_path}") and os.path.getsize("{full_path}") > 1024:
        print("HF_SUCCESS")
    else:
        print("HF_FAILED")
        sys.exit(1)

except Exception as e:
    print(f"HF_ERROR: {{e}}")
    sys.exit(1)
'''

                result = subprocess.run([
                    self.python_cmd, "-c", download_script
                ], capture_output=True, text=True, timeout=600)

                if result.returncode == 0 and "HF_SUCCESS" in result.stdout:
                    print("✅ HuggingFace Hub download successful")
                    return True
                else:
                    print(f"❌ HuggingFace Hub failed: {result.stderr}")
                    return False
            else:
                print("⚠️ URL not compatible with HuggingFace Hub")
                return False

        except Exception as e:
            print(f"❌ HuggingFace Hub error: {e}")
            return False

    def try_aria2c_download(self, url, full_path):
        """Try aria2c download (fastest for large files)"""
        try:
            print("🔄 Trying aria2c (16 connections)...")

            result = subprocess.run([
                "aria2c",
                "--max-connection-per-server=16",
                "--split=16",
                "--min-split-size=1M",
                "--continue=true",
                "--timeout=60",
                "--retry-wait=3",
                "--max-tries=3",
                "--out", os.path.basename(full_path),
                "--dir", os.path.dirname(full_path) or ".",
                url
            ], capture_output=True, text=True, timeout=1800)

            if result.returncode == 0 and os.path.exists(full_path):
                print("✅ aria2c download successful")
                return True
            else:
                print(f"❌ aria2c failed: {result.stderr}")
                return False

        except FileNotFoundError:
            print("❌ aria2c not found")
            return False
        except Exception as e:
            print(f"❌ aria2c error: {e}")
            return False

    def try_wget_download(self, url, full_path):
        """Try wget download"""
        try:
            print("🔄 Trying wget...")

            result = subprocess.run([
                "wget",
                "--continue",
                "--timeout=60",
                "--tries=3",
                "--retry-connrefused",
                "--waitretry=3",
                "-O", full_path,
                url
            ], capture_output=True, text=True, timeout=1800)

            if result.returncode == 0 and os.path.exists(full_path):
                print("✅ wget download successful")
                return True
            else:
                print(f"❌ wget failed: {result.stderr}")
                return False

        except FileNotFoundError:
            print("❌ wget not found")
            return False
        except Exception as e:
            print(f"❌ wget error: {e}")
            return False

    def try_urllib_download(self, url, full_path):
        """Try Python urllib download with progress"""
        try:
            print("🔄 Trying Python urllib...")

            def download_with_progress(url, filepath):
                try:
                    # Ensure directory exists
                    Path(filepath).parent.mkdir(parents=True, exist_ok=True)

                    # Download file
                    urllib.request.urlretrieve(url, filepath)

                    # Verify download
                    return os.path.exists(filepath) and os.path.getsize(filepath) > 1024
                except Exception as e:
                    print(f"urllib error: {e}")
                    return False

            if download_with_progress(url, full_path):
                print("✅ Python urllib download successful")
                return True
            else:
                print("❌ Python urllib download failed")
                return False

        except Exception as e:
            print(f"❌ Python urllib error: {e}")
            return False

    def download_essential_pack(self):
        """Download essential models and LoRAs for immediate functionality"""
        print("🎯 Downloading Essential Pack for WAN2GP Interface Population...")
        print("=" * 70)
        print("📋 This will populate:")
        print("   • Model dropdown with Wan 2.1 T2V models")
        print("   • Advanced tab LoRA checkboxes with speed enhancements")
        print("   • Required text encoders and VAE models")
        print("=" * 70)

        # Essential items in priority order
        essential_priority = [
            # Critical for basic functionality
            ("wan_vae_bf16", "essential"),     # Required VAE
            ("umt5_encoder", "essential"),     # Required text encoder
            ("wan_t2v_1_3b", "essential"),     # Main model for dropdown
            # Speed enhancements for LoRA checkboxes
            ("safe_forcing", "essential"),     # 2-step generation
            ("causvid", "essential"),          # Fast generation
            # Additional functionality
            ("wan_t2v_1_3b_fp32", "high"),     # I2V compatibility
            ("wan_vae_fp32", "high"),          # Alternative VAE
            ("accvid_t2v", "high"),            # T2V acceleration
            ("accvid_i2v", "high")             # I2V acceleration
        ]

        successful_downloads = 0
        total_items = len(essential_priority)

        for i, (item_id, priority) in enumerate(essential_priority, 1):
            config = self.all_downloads[item_id]
            print(f"\n[{i}/{total_items}] Processing {config['name']} (Priority: {priority})")
            print("=" * 60)

            if self.download_with_fallback_methods(config, item_id):
                successful_downloads += 1
                print(f"✅ {config['name']} downloaded successfully")
            else:
                print(f"❌ {config['name']} download failed")
                if priority == "essential":
                    print("⚠️ This is an essential file - WAN2GP functionality may be limited")

        success_rate = (successful_downloads / total_items) * 100

        print(f"\n📊 Essential Pack Download Summary:")
        print(f"Total items: {total_items}")
        print(f"Successful: {successful_downloads}")
        print(f"Failed: {total_items - successful_downloads}")
        print(f"Success rate: {success_rate:.1f}%")

        # Verify interface population
        self.verify_interface_population()

        if success_rate >= 80:
            print("🎉 Essential pack download completed successfully!")
            print("✅ WAN2GP interface should now show models and LoRAs")
            return True
        else:
            print("⚠️ Some downloads failed, but WAN2GP may still be functional")
            print("🔄 You can re-run this cell to retry failed downloads")
            return False

    def verify_interface_population(self):
        """Verify that downloaded files will populate the Gradio interface"""
        print(f"\n🔍 Verifying Interface Population...")

        # Check models for dropdown population
        model_files = list(Path("ckpts").glob("*.safetensors"))
        print(f"📄 Models in ckpts/ (will appear in model dropdown): {len(model_files)}")
        for model in model_files:
            print(f"   ✅ {model.name}")

        # Check LoRAs for checkbox population
        lora_files = list(Path("loras").glob("*.safetensors"))
        print(f"🎨 LoRAs in loras/ (will appear in Advanced tab): {len(lora_files)}")
        for lora in lora_files:
            print(f"   ✅ {lora.name}")

        # Check I2V LoRAs
        i2v_lora_files = list(Path("lorasi2v").glob("*.safetensors"))
        print(f"🖼️ I2V LoRAs in lorasi2v/: {len(i2v_lora_files)}")
        for lora in i2v_lora_files:
            print(f"   ✅ {lora.name}")

        # Check essential components
        encoder_files = list(Path("text_encoders").glob("*.safetensors"))
        print(f"📝 Text encoders (required): {len(encoder_files)}")

        vae_files = list(Path("vae").glob("*.safetensors"))
        print(f"🎭 VAE models (required): {len(vae_files)}")

        # Summary for interface readiness
        interface_ready = (len(model_files) >= 1 and len(encoder_files) >= 1 and
                          len(vae_files) >= 1 and len(lora_files) >= 2)

        if interface_ready:
            print(f"\n✅ Interface Population Verified!")
            print(f"🎉 WAN2GP will show:")
            print(f"   • {len(model_files)} model(s) in dropdown menu")
            print(f"   • {len(lora_files)} LoRA(s) in Advanced tab checkboxes")
            print(f"   • All required components for generation")
        else:
            print(f"\n⚠️ Interface may have limited options")
            print(f"💡 Ensure at least 1 model, 1 encoder, 1 VAE, and 2 LoRAs are downloaded")

        return interface_ready

# Create and execute the download system
downloader = WAN2GPCompleteDownloader()

# Display download interface
display(HTML(f"""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
           color: white; padding: 20px; border-radius: 10px; text-align: center;
           margin: 10px 0; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
    <h2>🎯 WAN2GP Essential Downloads</h2>
    <p><strong>Platform:</strong> {downloader.current_platform}</p>
    <p><strong>Python Command:</strong> {downloader.python_cmd}</p>
    <p>This will populate your Gradio interface with models and LoRAs</p>
</div>
"""))

print("🎯 WAN2GP Essential Download Options:")
print("=" * 60)
print("1. Essential Pack (Recommended) - ~8GB")
print("   • Wan T2V 1.3B model + FP32 variant")
print("   • Text encoder & VAE (required components)")
print("   • Speed enhancement LoRAs (Safe-Forcing, CausVid, AccVid)")
print("   • Populates model dropdown and LoRA checkboxes")
print("")
print("2. Skip Downloads")
print("   • Use if models already downloaded")
print("   • WAN2GP will auto-download minimal models on first use")
print("")

choice = input("🎯 Select option (1/2 or Enter for Essential): ").strip()

# Execute download based on choice
if choice == "2":
    print("⏭️ Skipping downloads - WAN2GP will auto-download models when needed")
    success = True
else:
    # Create directory structure first
    if not downloader.create_directory_structure():
        print("❌ Failed to create directory structure")
        success = False
    else:
        # Download essential pack
        success = downloader.download_essential_pack()

if success:
    print("\n🎉 Download system completed successfully!")
    print("✅ WAN2GP models and LoRAs should now appear in interface menus")
    print("🚀 Proceed to Cell 4 to launch WAN2GP with public Gradio link")
else:
    print("\n⚠️ Some downloads may have failed")
    print("🔄 You can re-run this cell to retry failed downloads")
    print("💡 WAN2GP may still work with auto-downloaded models")


🎯 WAN2GP Essential Download Options:
1. Essential Pack (Recommended) - ~8GB
   • Wan T2V 1.3B model + FP32 variant
   • Text encoder & VAE (required components)
   • Speed enhancement LoRAs (Safe-Forcing, CausVid, AccVid)
   • Populates model dropdown and LoRA checkboxes

2. Skip Downloads
   • Use if models already downloaded
   • WAN2GP will auto-download minimal models on first use

🎯 Select option (1/2 or Enter for Essential): 1
📁 Creating WAN2GP directory structure...
   ✅ ckpts/ - Main model files - populate model dropdown
   ✅ text_encoders/ - Text encoding models - required for generation
   ✅ vae/ - VAE models - required for generation
   ✅ vae_approx/ - Approximate VAE models
   ✅ clip_vision/ - CLIP vision models
   ✅ loras/ - General T2V LoRAs - populate LoRA checkboxes
   ✅ loras1.3B/ - 1.3B specific LoRAs
   ✅ loras14B/ - 14B specific LoRAs
   ✅ lorasi2v/ - Image-to-video LoRAs
   ✅ lorashunyuan/ - Hunyuan Video LoRAs
   ✅ lorashunyuani2v/ - Hunyuan I2V LoRAs
   ✅ loraslt

In [ ]:
#@title Cell (4) WAN2GP Launch - Enhanced with Public Gradio Link & Error Handling v5.0

import os
import subprocess
import signal
import time
import sys
from IPython.display import display, HTML

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        from IPython.display import display, Markdown
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Launch - Enhanced with Public Gradio Link & Error Handling")

class WAN2GPLauncher:
    def __init__(self):
        # Use global platform configuration from Cell 1
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        # CRITICAL ENFORCEMENT: Ensure venv for Google Colab per system instructions
        self.enforce_colab_venv()

        # Model configurations from documentation[3][14]
        self.model_options = {
            "1": {
                "flag": "--t2v-1-3B",
                "name": "Wan 2.1 Text2Video 1.3B",
                "vram": "6GB",
                "speed": "Fast",
                "description": "Fast generation, lower VRAM"
            },
            "2": {
                "flag": "--t2v-14B",
                "name": "Wan 2.1 Text2Video 14B",
                "vram": "12GB",
                "speed": "Slower",
                "description": "High quality, more VRAM"
            },
            "3": {
                "flag": "--i2v-1-3B",
                "name": "Wan Fun InP 1.3B",
                "vram": "6GB",
                "speed": "Fast",
                "description": "Image-to-video, fast"
            },
            "4": {
                "flag": "--i2v-14B",
                "name": "Wan 2.1 Image2Video 14B",
                "vram": "12GB",
                "speed": "Slower",
                "description": "High quality I2V"
            },
            "5": {
                "flag": "--vace-1-3B",
                "name": "Wan Vace 1.3B",
                "vram": "6GB",
                "speed": "Fast",
                "description": "ControlNet, motion transfer"
            },
            "6": {
                "flag": "--vace-14B",
                "name": "Wan Vace 14B",
                "vram": "12GB",
                "speed": "Slower",
                "description": "Advanced ControlNet features"
            }
        }

        # Performance configurations from documentation[5][13]
        self.performance_options = {
            "A": {
                "name": "Low VRAM Mode",
                "args": ["--profile", "4", "--attention", "sdpa"],
                "description": "Memory efficient, stable (RTX 10XX-20XX)"
            },
            "B": {
                "name": "High Performance Mode",
                "args": ["--compile", "--attention", "sage", "--profile", "3"],
                "description": "Fast generation (RTX 30XX-40XX)"
            },
            "C": {
                "name": "Emergency Fallback",
                "args": ["--attention", "sdpa", "--profile", "4", "--fp16"],
                "description": "Maximum compatibility"
            }
        }

    def detect_platform_fallback(self):
        """Fallback platform detection - ENFORCES VENV FOR COLAB per system instructions"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def enforce_colab_venv(self):
        """CRITICAL: Enforce venv usage for Google Colab per system instructions"""
        if hasattr(self, 'current_platform') and self.current_platform == "Google Colab":
            if not hasattr(self, 'use_venv') or not self.use_venv or self.python_cmd != ".venv/bin/python":
                print("🚨 ENFORCING: Google Colab MUST use venv per system instructions")
                self.use_venv = True
                self.pip_cmd = ".venv/bin/pip"
                self.python_cmd = ".venv/bin/python"
                print("✅ ENFORCED: Launch system using venv commands")
            else:
                print("✅ VERIFIED: Launch system correctly using venv for Colab")

    def setup_runtime_environment(self):
        """Setup runtime environment to prevent common errors"""
        print("🔧 Setting up runtime environment...")

        # Fix environment variables for headless operation
        env_fixes = {
            'XDG_RUNTIME_DIR': '/tmp/runtime-root',
            'ALSA_CARD': 'null',
            'PULSE_SERVER': 'unix:/dev/null',
            'MPLBACKEND': 'Agg',
            'MATPLOTLIB_BACKEND': 'Agg'
        }

        for key, value in env_fixes.items():
            os.environ[key] = value
            print(f"   ✅ {key} = {value}")

        # Create runtime directory
        os.makedirs('/tmp/runtime-root', exist_ok=True)

        print("✅ Runtime environment configured for headless operation")
        return True

    def show_launch_options(self):
        """Display launch options with public link information"""
        display(HTML(f"""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                   color: white; padding: 20px; border-radius: 10px; text-align: center;
                   margin: 10px 0; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
            <h2>🚀 WAN2GP Launch Options</h2>
            <p><strong>Platform:</strong> {self.current_platform}</p>
            <p><strong>Python Command:</strong> {self.python_cmd}</p>
            <p>Includes <strong>--share</strong> flag for public Gradio link generation</p>
        </div>
        """))

        print("🎯 WAN2GP Model Selection (with Public Gradio Link):")
        print("=" * 70)

        for key, config in self.model_options.items():
            print(f"{key}. {config['name']}")
            print(f"   VRAM: {config['vram']} | Speed: {config['speed']}")
            print(f"   {config['description']}")
            print()

        print("🔧 Performance Optimization Options:")
        print("=" * 40)
        for key, config in self.performance_options.items():
            print(f"{key}. {config['name']}")
            print(f"   {config['description']}")
            print()

        # Get user selections
        model_choice = input("🎯 Select model (1-6, or Enter for default Wan T2V 1.3B): ").strip()
        perf_choice = input("⚡ Select performance (A-C, or Enter for Low VRAM): ").strip().upper()

        # Process selections
        if model_choice in self.model_options:
            selected_model = self.model_options[model_choice]
            model_flag = selected_model["flag"]
            model_name = selected_model["name"]
        else:
            model_flag = "--t2v-1-3B"
            model_name = "Wan 2.1 Text2Video 1.3B (default)"

        if perf_choice in self.performance_options:
            perf_config = self.performance_options[perf_choice]
            perf_args = perf_config["args"]
            perf_name = perf_config["name"]
        else:
            perf_args = ["--profile", "4", "--attention", "sdpa"]
            perf_name = "Low VRAM Mode (default)"

        return model_flag, model_name, perf_args, perf_name

    def launch_wan2gp_with_public_link(self, model_flag, model_name, perf_args, perf_name):
        """Launch WAN2GP with public Gradio link and comprehensive error handling"""
        print("🚀 Launching WAN2GP with Public Gradio Link...")
        print("=" * 70)
        print(f"Platform: {self.current_platform}")
        print(f"Python Command: {self.python_cmd}")
        print(f"Selected Model: {model_name}")
        print(f"Performance Mode: {perf_name}")
        print("=" * 70)

        # Final enforcement check per system instructions
        if self.current_platform == "Google Colab" and not self.use_venv:
            print("🚨 FATAL: Google Colab not using venv per system instructions - ABORTING")
            return False

        # Setup runtime environment
        self.setup_runtime_environment()

        # Verify we're in the correct directory
        if not os.path.exists("wgp.py"):
            print("❌ wgp.py not found. Make sure you're in the Wan2GP directory.")
            print(f"📍 Current directory: {os.getcwd()}")
            return False

        # Build launch command with public link support from CLI documentation[5][13]
        base_cmd = [
            self.python_cmd,
            "wgp.py",
            model_flag,
            "--share",        # Creates public HuggingFace URL per documentation[5]
            "--listen",       # Make server accessible on network
            "--server-port", "7860"
        ]

        # Add performance arguments
        base_cmd.extend(perf_args)

        # Add stability arguments from troubleshooting guide[8][16]
        stability_args = [
            "--verbose", "1"  # Moderate verbosity for monitoring
        ]
        base_cmd.extend(stability_args)

        print(f"🎯 Launch command: {' '.join(base_cmd)}")
        print()
        print("🔄 Starting WAN2GP...")
        print("🌐 Local interface: http://localhost:7860")
        print("🔗 Public Gradio link will be generated and displayed below")
        print("⏹️ Press Ctrl+C in this cell to stop WAN2GP")
        print("=" * 70)

        try:
            # Launch WAN2GP with real-time output monitoring
            process = subprocess.Popen(
                base_cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1
            )

            gradio_link_found = False
            start_time = time.time()
            output_lines = []

            print("📋 WAN2GP Output:")
            print("-" * 50)

            # Monitor output for important information
            while True:
                output = process.stdout.readline()
                if output == '' and process.poll() is not None:
                    break

                if output:
                    line = output.strip()
                    print(line)
                    output_lines.append(line)

                    # Detect and highlight public Gradio link
                    if "Running on public URL:" in line and not gradio_link_found:
                        gradio_link_found = True
                        # Extract URL
                        url = line.split("Running on public URL:")[-1].strip()

                        # Display prominent link notification
                        display(HTML(f"""
                        <div style="background: linear-gradient(90deg, #00ff41, #00d4ff);
                                    padding: 20px; border-radius: 15px; margin: 15px 0;
                                    border: 3px solid #00ff41; text-align: center;
                                    box-shadow: 0 8px 25px rgba(0,255,65,0.3);">
                            <h2 style="color: #000; margin: 0; font-size: 24px;">
                                🌐 PUBLIC GRADIO LINK READY!
                            </h2>
                            <p style="color: #000; margin: 10px 0; font-size: 18px;">
                                <strong>Click here to access WAN2GP remotely:</strong>
                            </p>
                            <div style="background: white; padding: 15px; border-radius: 10px;
                                        margin: 10px 0; border: 2px solid #000;">
                                <a href="{url}" target="_blank"
                                   style="color: #000; font-weight: bold; font-size: 20px;
                                          text-decoration: none;">
                                    {url}
                                </a>
                            </div>
                            <p style="color: #000; margin: 5px 0; font-size: 14px;">
                                🕒 Link expires in 72 hours • 🔗 Right-click to copy
                            </p>
                        </div>
                        """))

                    # Monitor for successful startup indicators
                    if "Running on local URL:" in line:
                        print("✅ Local server started successfully")

                    # Monitor for model loading
                    if "Loading model" in line or "Model loaded" in line:
                        print("📦 Model loading detected...")

                    # Filter harmless warnings but continue execution
                    harmless_warnings = [
                        "xdg_runtime_dir", "alsa lib", "unknown pcm",
                        "no such file or directory", "switching to fp16"
                    ]

                    if any(warning in line.lower() for warning in harmless_warnings):
                        continue  # Don't treat as errors

                # Timeout check (10 minutes for initial startup)
                if time.time() - start_time > 600:
                    print("⚠️ Startup timeout reached (10 minutes)")
                    print("🔍 Check output above for any error messages")
                    break

            # Final status check
            if not gradio_link_found:
                print("\n⚠️ Public Gradio link not detected in output")
                print("🔍 WAN2GP may still be accessible locally at: http://localhost:7860")
                print("💡 Check the output above for any error messages")

            # Keep process running until interrupted
            try:
                print("\n⏸️ WAN2GP is running. Press Ctrl+C to stop...")
                process.wait()
            except KeyboardInterrupt:
                print("\n⏹️ Stopping WAN2GP...")
                process.terminate()
                # Give process time to clean up
                try:
                    process.wait(timeout=5)
                except subprocess.TimeoutExpired:
                    process.kill()
                print("✅ WAN2GP stopped successfully")

        except Exception as e:
            print(f"❌ Error launching WAN2GP: {e}")
            print("\n🔧 Troubleshooting suggestions:")
            print("1. Verify models were downloaded in Cell 3")
            print("2. Check that wgp.py exists in current directory")
            print("3. Try the emergency fallback command below")
            print("\n🆘 Emergency fallback command:")
            print(f"{self.python_cmd} wgp.py --t2v-1-3B --attention sdpa --profile 4")
            return False

        return gradio_link_found

# Create launcher instance
launcher = WAN2GPLauncher()

# Show options and get user selections
model_flag, model_name, perf_args, perf_name = launcher.show_launch_options()

# Launch WAN2GP with selected configuration
success = launcher.launch_wan2gp_with_public_link(model_flag, model_name, perf_args, perf_name)

if success:
    print("\n🎉 WAN2GP launch completed successfully!")
else:
    print("\n⚠️ Launch encountered issues - check output above for details")


✅ VERIFIED: Launch system correctly using venv for Colab


🎯 WAN2GP Model Selection (with Public Gradio Link):
1. Wan 2.1 Text2Video 1.3B
   VRAM: 6GB | Speed: Fast
   Fast generation, lower VRAM

2. Wan 2.1 Text2Video 14B
   VRAM: 12GB | Speed: Slower
   High quality, more VRAM

3. Wan Fun InP 1.3B
   VRAM: 6GB | Speed: Fast
   Image-to-video, fast

4. Wan 2.1 Image2Video 14B
   VRAM: 12GB | Speed: Slower
   High quality I2V

5. Wan Vace 1.3B
   VRAM: 6GB | Speed: Fast
   ControlNet, motion transfer

6. Wan Vace 14B
   VRAM: 12GB | Speed: Slower
   Advanced ControlNet features

🔧 Performance Optimization Options:
A. Low VRAM Mode
   Memory efficient, stable (RTX 10XX-20XX)

B. High Performance Mode
   Fast generation (RTX 30XX-40XX)

C. Emergency Fallback
   Maximum compatibility



In [ ]:
#@title Cell (1) - WAN2GP Comprehensive Diagnostic and Auto-Repair System (Latest v6.2 Compatible)

import sys
import os
import subprocess
import shutil
import time
import json
from pathlib import Path
from IPython.display import display, HTML, Markdown
import importlib.util

class WAN2GPDiagnosticAndRepair:
    def __init__(self):
        self.platform = self.detect_platform()
        self.pip_cmd, self.python_cmd, self.use_venv = self.get_platform_commands()
        self.issues_found = []
        self.repairs_applied = []
        self.gpu_info = {}

    def detect_platform(self):
        """Enhanced platform detection with comprehensive indicators"""
        # Lightning AI detection - multiple indicators for reliability
        lightning_indicators = [
            "lightning" in str(sys.executable).lower(),
            "teamspace-studios" in os.getcwd(),
            "LIGHTNING_CLOUDSPACE_HOST" in os.environ,
            "LIGHTNING_CLOUDSPACE_ID" in os.environ,
            "/commands/python" in str(sys.executable),
            "/home/zeus/miniconda3/envs/cloudspace" in str(sys.executable),
            os.path.exists("/teamspace"),
            os.path.exists("/commands")
        ]

        # Google Colab detection
        colab_indicators = [
            "google.colab" in sys.modules,
            "/content" in os.getcwd()
        ]

        # Vast.AI detection
        vast_indicators = [
            "VAST_CONTAINER_LABEL" in os.environ,
            "/workspace" in os.getcwd(),
            "vast" in os.environ.get("HOSTNAME", "").lower()
        ]

        if any(lightning_indicators):
            return "Lightning AI"
        elif any(colab_indicators):
            return "Google Colab"
        elif any(vast_indicators):
            return "Vast.AI/Generic"
        else:
            return "Vast.AI/Generic"

    def get_platform_commands(self):
        """Get platform-specific pip and python commands"""
        if self.platform == "Lightning AI":
            return "pip", "python", False  # (pip_cmd, python_cmd, use_venv)
        elif self.platform == "Google Colab":
            return ".venv/bin/pip", ".venv/bin/python", True
        else:  # Vast.AI/Generic
            return ".venv/bin/pip", ".venv/bin/python", True

    def run_command_safely(self, command, description, timeout=60):
        """Execute command with comprehensive error handling"""
        try:
            result = subprocess.run(
                command, capture_output=True, text=True, timeout=timeout, shell=isinstance(command, str)
            )
            if result.returncode == 0:
                return True, result.stdout
            else:
                return False, result.stderr
        except subprocess.TimeoutExpired:
            return False, f"Timeout after {timeout}s"
        except Exception as e:
            return False, str(e)

    def check_gpu_compatibility(self):
        """Comprehensive GPU detection and compatibility check"""
        print("🔍 Checking GPU compatibility...")

        # Check NVIDIA GPU presence
        success, output = self.run_command_safely("nvidia-smi", "GPU Detection")
        if not success:
            self.issues_found.append("No NVIDIA GPU detected or nvidia-smi not available")
            return False

        # Parse GPU info
        try:
            # Extract GPU name and VRAM from nvidia-smi output
            lines = output.split('\n')
            for line in lines:
                if 'RTX' in line or 'GTX' in line or 'Tesla' in line or 'A100' in line:
                    gpu_name = line.split('|')[1].strip() if '|' in line else "Unknown GPU"
                    self.gpu_info['name'] = gpu_name
                    break

            # Check VRAM
            for line in lines:
                if 'MiB' in line and '/' in line:
                    vram_info = [part for part in line.split() if 'MiB' in part]
                    if len(vram_info) >= 2:
                        total_vram = int(vram_info[-1].replace('MiB', ''))
                        self.gpu_info['vram_mb'] = total_vram
                        self.gpu_info['vram_gb'] = total_vram / 1024
                        break

            print(f"✅ GPU detected: {self.gpu_info.get('name', 'Unknown')}")
            print(f"✅ VRAM: {self.gpu_info.get('vram_gb', 0):.1f}GB")

            # Check VRAM adequacy
            vram_gb = self.gpu_info.get('vram_gb', 0)
            if vram_gb < 6:
                self.issues_found.append(f"Low VRAM detected ({vram_gb:.1f}GB). Minimum 6GB recommended.")

            return True

        except Exception as e:
            self.issues_found.append(f"GPU info parsing failed: {str(e)}")
            return False

    def check_cuda_pytorch_compatibility(self):
        """Check CUDA and PyTorch compatibility"""
        print("🔍 Checking CUDA and PyTorch compatibility...")

        # Check CUDA version
        success, cuda_output = self.run_command_safely("nvcc --version", "CUDA Version Check")
        if not success:
            success, cuda_output = self.run_command_safely("nvidia-smi", "CUDA Runtime Check")

        # Check PyTorch installation
        try:
            import torch
            pytorch_version = torch.__version__
            cuda_available = torch.cuda.is_available()

            print(f"✅ PyTorch version: {pytorch_version}")
            print(f"✅ CUDA available in PyTorch: {cuda_available}")

            if not cuda_available:
                self.issues_found.append("PyTorch cannot detect CUDA")
                return False

            # Check for version compatibility
            if "50" in self.gpu_info.get('name', '') and not pytorch_version.startswith('2.7'):
                self.issues_found.append("RTX 50XX series requires PyTorch 2.7.0 or newer")

            return True

        except ImportError:
            self.issues_found.append("PyTorch not installed")
            return False
        except Exception as e:
            self.issues_found.append(f"PyTorch check failed: {str(e)}")
            return False

    def check_python_version(self):
        """Check Python version compatibility"""
        print("🔍 Checking Python version...")

        python_version = sys.version_info
        version_string = f"{python_version.major}.{python_version.minor}.{python_version.micro}"
        print(f"✅ Python version: {version_string}")

        if python_version.major != 3 or python_version.minor != 10:
            self.issues_found.append(f"Python {version_string} detected. Python 3.10.9 recommended for best compatibility.")

        return True

    def check_wan2gp_installation(self):
        """Check WAN2GP installation and repository"""
        print("🔍 Checking WAN2GP installation...")

        wan2gp_paths = ["Wan2GP", "wan2gp", "WAN2GP", "../Wan2GP", "../wan2gp"]
        wan2gp_found = False

        for path in wan2gp_paths:
            if os.path.exists(path):
                wan2gp_found = True
                wgp_py_path = os.path.join(path, "wgp.py")
                if os.path.exists(wgp_py_path):
                    print(f"✅ WAN2GP found at: {path}")
                    return True
                break

        if not wan2gp_found:
            self.issues_found.append("WAN2GP repository not found")
            return False

        self.issues_found.append("WAN2GP repository found but wgp.py missing")
        return False

    def check_dependencies(self):
        """Check critical dependencies"""
        print("🔍 Checking critical dependencies...")

        critical_deps = {
            'torch': 'PyTorch',
            'torchvision': 'TorchVision',
            'gradio': 'Gradio',
            'transformers': 'Transformers',
            'accelerate': 'Accelerate',
            'diffusers': 'Diffusers'
        }

        missing_deps = []
        for dep, name in critical_deps.items():
            try:
                importlib.import_module(dep)
                print(f"✅ {name} installed")
            except ImportError:
                missing_deps.append(dep)
                print(f"❌ {name} missing")

        if missing_deps:
            self.issues_found.append(f"Missing dependencies: {', '.join(missing_deps)}")
            return False

        return True

    def check_attention_mechanisms(self):
        """Check available attention mechanisms"""
        print("🔍 Checking attention mechanisms...")

        attention_status = {}

        # Check Triton
        try:
            import triton
            attention_status['triton'] = f"✅ Triton {triton.__version__}"
        except ImportError:
            attention_status['triton'] = "❌ Triton not available"

        # Check SageAttention
        try:
            import sageattention
            attention_status['sage'] = "✅ SageAttention available"
        except ImportError:
            attention_status['sage'] = "❌ SageAttention not available"

        # Check Flash Attention
        try:
            import flash_attn
            attention_status['flash'] = "✅ Flash Attention available"
        except ImportError:
            attention_status['flash'] = "❌ Flash Attention not available"

        for mech, status in attention_status.items():
            print(f"  {status}")

        # Recommend fallback if advanced attention not available
        if all("❌" in status for status in attention_status.values()):
            self.issues_found.append("No advanced attention mechanisms available. Will use SDPA fallback.")

        return True

    def check_memory_and_performance(self):
        """Check memory and performance configuration"""
        print("🔍 Checking memory and performance...")

        # Check available system RAM
        try:
            import psutil
            ram_gb = psutil.virtual_memory().total / (1024**3)
            print(f"✅ System RAM: {ram_gb:.1f}GB")

            if ram_gb < 16:
                self.issues_found.append(f"Low system RAM ({ram_gb:.1f}GB). 16GB+ recommended for best performance.")
        except ImportError:
            print("⚠️ Cannot check system RAM (psutil not available)")

        # Check disk space
        try:
            disk_usage = shutil.disk_usage(os.getcwd())
            free_gb = disk_usage.free / (1024**3)
            print(f"✅ Free disk space: {free_gb:.1f}GB")

            if free_gb < 20:
                self.issues_found.append(f"Low disk space ({free_gb:.1f}GB). 20GB+ recommended.")
        except Exception:
            print("⚠️ Cannot check disk space")

        return True

    def auto_repair_pytorch(self):
        """Auto-repair PyTorch installation"""
        print("🔧 Attempting PyTorch repair...")

        # Determine correct PyTorch version based on GPU
        if "50" in self.gpu_info.get('name', ''):
            # RTX 50XX series
            pytorch_cmd = [
                self.pip_cmd, "install", "--upgrade",
                "torch==2.7.0", "torchvision", "torchaudio",
                "--index-url", "https://download.pytorch.org/whl/test/cu128"
            ]
        else:
            # RTX 10XX-40XX series
            pytorch_cmd = [
                self.pip_cmd, "install", "--upgrade",
                "torch==2.6.0", "torchvision", "torchaudio",
                "--index-url", "https://download.pytorch.org/whl/test/cu124"
            ]

        success, output = self.run_command_safely(pytorch_cmd, "PyTorch Installation", timeout=300)
        if success:
            self.repairs_applied.append("PyTorch installation repaired")
            return True
        else:
            print(f"❌ PyTorch repair failed: {output}")
            return False

    def auto_repair_dependencies(self):
        """Auto-repair missing dependencies"""
        print("🔧 Installing missing dependencies...")

        # Install core requirements
        requirements_cmd = [self.pip_cmd, "install", "-r", "requirements.txt"]
        success, output = self.run_command_safely(requirements_cmd, "Dependencies Installation", timeout=300)

        if success:
            self.repairs_applied.append("Dependencies installed")
            return True
        else:
            # Fallback: install critical packages individually
            critical_packages = [
                "gradio>=4.0.0", "transformers", "accelerate", "diffusers",
                "opencv-python", "Pillow", "numpy", "scipy"
            ]

            for package in critical_packages:
                cmd = [self.pip_cmd, "install", package]
                success, _ = self.run_command_safely(cmd, f"Installing {package}", timeout=60)
                if success:
                    print(f"✅ Installed {package}")

            self.repairs_applied.append("Critical dependencies installed individually")
            return True

    def auto_repair_wan2gp_repo(self):
        """Auto-repair WAN2GP repository"""
        print("🔧 Cloning WAN2GP repository...")

        repo_url = "https://github.com/deepbeepmeep/Wan2GP.git"
        clone_cmd = ["git", "clone", "--depth", "1", repo_url]

        success, output = self.run_command_safely(clone_cmd, "Repository Clone", timeout=120)
        if success:
            self.repairs_applied.append("WAN2GP repository cloned")
            return True
        else:
            print(f"❌ Repository clone failed: {output}")
            return False

    def auto_repair_attention_mechanisms(self):
        """Auto-repair attention mechanisms"""
        print("🔧 Installing performance optimizations...")

        repairs = []

        # Install Triton for Windows
        if os.name == 'nt':  # Windows
            triton_cmd = [self.pip_cmd, "install", "triton-windows"]
            success, _ = self.run_command_safely(triton_cmd, "Triton Installation", timeout=120)
            if success:
                repairs.append("Triton (Windows)")

        # Install SageAttention
        sage_cmd = [self.pip_cmd, "install", "sageattention>=1.0.6"]
        success, _ = self.run_command_safely(sage_cmd, "SageAttention Installation", timeout=120)
        if success:
            repairs.append("SageAttention")

        if repairs:
            self.repairs_applied.append(f"Installed: {', '.join(repairs)}")
            return True

        return False

    def generate_optimized_launch_commands(self):
        """Generate optimized launch commands based on hardware"""
        print("\n🚀 Generating optimized launch commands...")

        vram_gb = self.gpu_info.get('vram_gb', 8)
        gpu_name = self.gpu_info.get('name', 'Unknown')

        commands = {}

        # Base command components
        base_cmd = "python wgp.py"

        if vram_gb < 8:
            # Low VRAM setup
            commands['Low VRAM (6-8GB)'] = f"{base_cmd} --t2v-1-3B --attention sdpa --profile 4 --teacache 1.5"
        elif vram_gb < 12:
            # Medium VRAM setup
            commands['Medium VRAM (8-12GB)'] = f"{base_cmd} --t2v-14B --attention sage --profile 4 --teacache 2.0"
        else:
            # High VRAM setup
            commands['High VRAM (12GB+)'] = f"{base_cmd} --t2v-14B --attention sage2 --profile 3 --compile --teacache 2.0"

        # GPU-specific optimizations
        if "10" in gpu_name or "20" in gpu_name:
            commands['RTX 10XX/20XX Optimized'] = f"{base_cmd} --attention sdpa --profile 4 --teacache 1.5"
        elif "30" in gpu_name or "40" in gpu_name:
            commands['RTX 30XX/40XX Optimized'] = f"{base_cmd} --compile --attention sage --profile 3 --teacache 2.0"
        elif "50" in gpu_name:
            commands['RTX 50XX Optimized'] = f"{base_cmd} --attention sage --profile 4 --fp16"

        # Fallback command
        commands['Safe Fallback'] = f"{base_cmd} --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16"

        # Debug command
        commands['Debug Mode'] = f"{base_cmd} --verbose 2 --check-loras --attention sdpa --profile 4"

        return commands

    def run_full_diagnostic(self):
        """Run complete diagnostic and repair sequence"""
        print("🏥 WAN2GP Comprehensive Diagnostic and Auto-Repair System")
        print("=" * 60)
        print(f"🖥️ Platform: {self.platform}")
        print(f"🐍 Python: {self.python_cmd}")
        print(f"📦 Pip: {self.pip_cmd}")
        print(f"🔧 Virtual Environment: {'Yes' if self.use_venv else 'No'}")
        print("=" * 60)

        # Run all diagnostics
        checks = [
            self.check_python_version,
            self.check_gpu_compatibility,
            self.check_cuda_pytorch_compatibility,
            self.check_wan2gp_installation,
            self.check_dependencies,
            self.check_attention_mechanisms,
            self.check_memory_and_performance
        ]

        print("\n📋 Running Diagnostics...")
        for check in checks:
            try:
                check()
            except Exception as e:
                print(f"❌ Diagnostic error: {str(e)}")
                self.issues_found.append(f"Diagnostic error: {str(e)}")

        # Auto-repair if issues found
        if self.issues_found:
            print(f"\n⚠️ Found {len(self.issues_found)} issues:")
            for i, issue in enumerate(self.issues_found, 1):
                print(f"  {i}. {issue}")

            print(f"\n🔧 Attempting automatic repairs...")

            # Apply repairs based on issues found
            if any("PyTorch" in issue for issue in self.issues_found):
                self.auto_repair_pytorch()

            if any("dependencies" in issue.lower() for issue in self.issues_found):
                self.auto_repair_dependencies()

            if any("repository" in issue.lower() for issue in self.issues_found):
                self.auto_repair_wan2gp_repo()

            if any("attention" in issue.lower() for issue in self.issues_found):
                self.auto_repair_attention_mechanisms()

        # Generate launch commands
        commands = self.generate_optimized_launch_commands()

        # Final report
        self.display_final_report(commands)

    def display_final_report(self, commands):
        """Display comprehensive final report"""

        # Create styled HTML report
        html_report = f"""
        <div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
            <h2>🏥 WAN2GP Diagnostic Report</h2>
            <div style="background-color: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>📊 System Status</h3>
                <p><strong>Platform:</strong> {self.platform}</p>
                <p><strong>GPU:</strong> {self.gpu_info.get('name', 'Unknown')} ({self.gpu_info.get('vram_gb', 0):.1f}GB VRAM)</p>
                <p><strong>Issues Found:</strong> {len(self.issues_found)}</p>
                <p><strong>Repairs Applied:</strong> {len(self.repairs_applied)}</p>
            </div>
        </div>
        """

        if self.repairs_applied:
            html_report += f"""
            <div style="background-color: #28a745; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>✅ Repairs Applied Successfully:</h3>
                <ul>
            """
            for repair in self.repairs_applied:
                html_report += f"<li>{repair}</li>"
            html_report += "</ul></div>"

        if self.issues_found and not self.repairs_applied:
            html_report += f"""
            <div style="background-color: #dc3545; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>⚠️ Unresolved Issues:</h3>
                <ul>
            """
            for issue in self.issues_found:
                html_report += f"<li>{issue}</li>"
            html_report += "</ul></div>"

        display(HTML(html_report))

        # Display optimized commands
        print("\n🚀 Recommended Launch Commands:")
        print("=" * 50)
        for name, command in commands.items():
            print(f"\n{name}:")
            print(f"  {command}")

        print("\n" + "=" * 50)
        print("✅ Diagnostic complete! Use the appropriate command above to launch WAN2GP.")
        if self.issues_found and not self.repairs_applied:
            print("⚠️  Some issues require manual intervention. Check the report above.")

# Execute diagnostic system
diagnostic_system = WAN2GPDiagnosticAndRepair()
diagnostic_system.run_full_diagnostic()


In [ ]:
#@title Cell (2) - Quick Diagnostic Runner and Emergency Repair Tools (Latest v6.2)

import subprocess
import sys
import os
from IPython.display import display, HTML

class QuickDiagnosticRunner:
    def __init__(self):
        self.platform = self.detect_platform()
        self.pip_cmd, self.python_cmd = self.get_platform_commands()

    def detect_platform(self):
        """Quick platform detection"""
        if "google.colab" in sys.modules:
            return "Google Colab"
        elif any(indicator in str(sys.executable).lower() for indicator in ["lightning", "teamspace"]):
            return "Lightning AI"
        else:
            return "Generic/Vast.AI"

    def get_platform_commands(self):
        """Get platform-specific commands"""
        if self.platform == "Lightning AI":
            return "pip", "python"
        else:
            return ".venv/bin/pip" if os.path.exists(".venv") else "pip", ".venv/bin/python" if os.path.exists(".venv") else "python"

    def run_cmd(self, cmd, timeout=30):
        """Execute command safely"""
        try:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
            return result.returncode == 0, result.stdout, result.stderr
        except subprocess.TimeoutExpired:
            return False, "", "Timeout"
        except Exception as e:
            return False, "", str(e)

    def emergency_pytorch_fix(self):
        """Emergency PyTorch installation fix"""
        print("🚨 Emergency PyTorch Fix...")

        # Detect GPU generation for correct PyTorch version
        success, gpu_info, _ = self.run_cmd("nvidia-smi")

        if "RTX 50" in gpu_info:
            pytorch_url = "https://download.pytorch.org/whl/test/cu128"
            torch_version = "torch==2.7.0"
        else:
            pytorch_url = "https://download.pytorch.org/whl/test/cu124"
            torch_version = "torch==2.6.0"

        cmd = f"{self.pip_cmd} install --upgrade {torch_version} torchvision torchaudio --index-url {pytorch_url}"
        success, stdout, stderr = self.run_cmd(cmd, timeout=300)

        if success:
            print("✅ PyTorch emergency fix applied")
            return True
        else:
            print(f"❌ PyTorch fix failed: {stderr}")
            return False

    def emergency_dependency_fix(self):
        """Emergency dependency installation"""
        print("🚨 Emergency Dependency Fix...")

        essential_packages = [
            "gradio>=4.0.0", "transformers", "accelerate", "diffusers",
            "opencv-python", "Pillow", "numpy", "scipy", "psutil"
        ]

        for package in essential_packages:
            cmd = f"{self.pip_cmd} install {package}"
            success, _, _ = self.run_cmd(cmd, timeout=60)
            print("✅" if success else "❌", package)

        print("✅ Emergency dependencies installed")

    def quick_system_check(self):
        """Quick system health check"""
        checks = {}

        # GPU Check
        success, output, _ = self.run_cmd("nvidia-smi")
        checks['GPU'] = "✅ Available" if success and "RTX" in output else "❌ Issue detected"

        # PyTorch Check
        try:
            import torch
            checks['PyTorch'] = f"✅ {torch.__version__}" if torch.cuda.is_available() else "❌ CUDA not available"
        except ImportError:
            checks['PyTorch'] = "❌ Not installed"

        # WAN2GP Check
        wan_exists = any(os.path.exists(path) for path in ["Wan2GP/wgp.py", "wan2gp/wgp.py", "WAN2GP/wgp.py"])
        checks['WAN2GP'] = "✅ Found" if wan_exists else "❌ Missing"

        # Dependencies Check
        try:
            import gradio, transformers, accelerate, diffusers
            checks['Dependencies'] = "✅ Core packages available"
        except ImportError:
            checks['Dependencies'] = "❌ Missing packages"

        return checks

    def generate_emergency_commands(self):
        """Generate emergency launch commands"""
        commands = {
            "Ultra Safe Mode": f"{self.python_cmd} wgp.py --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16",
            "Memory Emergency": f"{self.python_cmd} wgp.py --t2v-1-3B --profile 5 --perc-reserved-mem-max 0.2",
            "Debug Mode": f"{self.python_cmd} wgp.py --verbose 2 --attention sdpa --profile 4",
            "Network Share": f"{self.python_cmd} wgp.py --listen --server-port 7861 --attention sdpa"
        }
        return commands

    def run_quick_diagnostic(self):
        """Run quick diagnostic and provide emergency options"""

        print(f"⚡ Quick Diagnostic - Platform: {self.platform}")
        print("=" * 40)

        # Quick system check
        checks = self.quick_system_check()

        issues = []
        for component, status in checks.items():
            print(f"{component}: {status}")
            if "❌" in status:
                issues.append(component)

        # Emergency repairs
        if issues:
            print(f"\n🚨 {len(issues)} issues detected. Applying emergency fixes...")

            if "PyTorch" in issues:
                self.emergency_pytorch_fix()

            if "Dependencies" in issues:
                self.emergency_dependency_fix()

            if "WAN2GP" in issues:
                print("🔧 Cloning WAN2GP repository...")
                success, _, _ = self.run_cmd("git clone --depth 1 https://github.com/deepbeepmeep/Wan2GP.git", timeout=120)
                print("✅ Repository cloned" if success else "❌ Clone failed")

        # Generate emergency commands
        commands = self.generate_emergency_commands()

        # Display results
        html_display = f"""
        <div style="background: linear-gradient(45deg, #ff6b6b, #feca57); color: white;
                    padding: 15px; border-radius: 8px; margin: 10px 0;">
            <h3>⚡ Quick Diagnostic Results</h3>
            <p><strong>Platform:</strong> {self.platform}</p>
            <p><strong>Issues Found:</strong> {len(issues)}</p>
            <p><strong>Status:</strong> {'🚨 Needs Attention' if issues else '✅ System Ready'}</p>
        </div>
        """

        display(HTML(html_display))

        print("\n🚀 Emergency Launch Commands:")
        print("-" * 40)
        for name, command in commands.items():
            print(f"\n{name}:")
            print(f"  {command}")

        print("\n" + "=" * 40)
        print("⚡ Quick diagnostic complete!")

        if not issues:
            print("✅ System appears healthy. Try the Ultra Safe Mode command first.")
        else:
            print("🚨 Emergency fixes applied. Test with Ultra Safe Mode.")

# Execute quick diagnostic
quick_runner = QuickDiagnosticRunner()
quick_runner.run_quick_diagnostic()
